In [32]:
import pandas as pd
import numpy as np

In [64]:
df = pd.read_excel('data.xlsx')

## Methods

We only care about time and activity for this analysis. We have to convert these two columns into the right representation for entering an HSMM. 

In [22]:
# activity mapping. we map activities to a 3-state representation - new, revisit and revise
# 'new' means the student moved to look at a new problem, we code it as state 0
# 'revisit' means they moved to look at a problem they have seen before, but haven't answered yet, we code it as state 1
# 'revise' means they moved to look at a problem they have already marked an answer for before, we code it as state 2

df.loc[df['Activity'].str.contains('first time'),'Activity'] = 0

df.loc[df['Activity'].str.contains('unanswered question', na=False),'Activity'] = 1

df.loc[df['Activity'].str.contains('answered question', na=False),'Activity'] = 2
df.loc[df['Activity'].str.contains('marked question', na=False),'Activity'] = 2

In [6]:
# time fix

df = df.assign(Logged_Time=pd.to_datetime(df.Logged_Time))

In [73]:
student_list = df.Roll_Number.unique()

In [150]:
new_df = pd.DataFrame(columns = ['id', 'states', 'durations'])

In [152]:

for student in student_list:

    start_index = df.loc[(df.Candidate_Status == 'Exam Started') & (df.Roll_Number == student)].index.values[0]

    id = df.loc[(df.Roll_Number == student) & df.Activity.isin([0,1,2])]['Roll_Number'] 
    states = df.loc[(df.Roll_Number == student) & df.Activity.isin([0,1,2])]['Activity'] 
    session_times = (df.loc[(df.Roll_Number == student) & df.Activity.isin([0,1,2])]['Logged_Time'] - df.loc[df.Roll_Number == student]['Logged_Time'][start_index]).dt.seconds

    durations = abs(session_times.diff(periods=-1))
    durations.iloc[-1] = 5
    new_df = new_df.append(pd.DataFrame({'id': id, 'states': states, 'durations': durations}).reset_index().drop('index',axis=1), sort=False)

In [162]:
new_df.to_csv('hsmm_data.csv')